In [ ]:
from geocoding_utils import *
from NTDFacilitiesInventory import *
import folium
from shared_utils import catalog_utils
import datetime as dt

In [ ]:
catalog = catalog_utils.get_catalog("shared_data_catalog")
states = catalog.us_states.read()
california = states.loc[states.STATE_ABBR.isin(["CA"])].copy()
california

#### Run the below cell to get raw results from the NTD, geocode them, and save the results in GCS. This will be billed once Google Cloud free tier limits are met, so should be used on a minimal sample

In [ ]:
# Load the NTD facilities inventory from the NTD, geocode, and save to GCS
timestamp = dt.datetime.now(dt.timezone.utc).isoformat()
output_file_uri_used = f"gs://calitp-analytics-data/data-analyses/ntd/ntd_transit_facilities/facilities_inventory_{timestamp}.geojson"
print(output_file_uri_used)
with open("most_recent_gcs_uri", "w") as f:
    f.write(output_file_uri_used)
ntd = NTDFacilitiesInventory(
    address_filter={"State": "CA"}, 
    geospatial_filter=california, 
    output_file_uri=output_file_uri_used,
)

#### Run the below cell to load the saved results from GCS

In [ ]:
# Load the NTD facilities inventory from the NTD (should not require geocoding)
with open("most_recent_gcs_uri") as f:
    read_uri = f.read()
ntd = NTDFacilitiesInventory(url=read_uri, attempt_geocoding=False)

#### Save the results as a shapefile in GCS (a geojson is created automatically)

In [ ]:
with open("most_recent_gcs_uri") as f:
    read_uri = f.read()
shapefile_path = read_uri[:-8].replace(".", "_") + ".zip"
ntd.save_shapefile(shapefile_path)

#### Visualize the results in a Folium Map

In [ ]:
m = folium.Map(location=[36.8790073,-119.8346849], zoom_start=6)
ntd.folium_geojson().add_to(m)
m